In [243]:
import os
import numpy as np
import torch
import pandas as pd
import yaml

def one_hot(i):
    a=[0 for ii in range(7)]
    a[i]=1
    return a


def get_data_train():
    label_transform = {'neutral': 0, 'surprise': 1, 'fear': 2, 'sadness': 3, 'joy': 4, 'disgust': 5, 'anger': 6}
    data_dict = {}
    csv_path_train ='C:/Users/faustineljc/Desktop/train_mfcc'
    csv_files_train = os.listdir(csv_path_train)
    #print(csv_files_train)
    csv_files_train.sort(key=lambda x:(int(x.split("_")[0][3:]), int(x.split('_')[1][3:-4])))
    for i in range(len(csv_files_train)):
        csv_files_train[i]=csv_files_train[i][:-4]
    with open('C:/Users/faustineljc/Desktop/melddatasets.yaml', 'rb') as stream:
        try:
            # print(yaml.safe_load(stream))
            output = yaml.safe_load(stream)
            # print(output)
            # print(len(output), type(output))  # len 3, type dict
            # print(output.keys())  # dict_keys(['dev', 'test', 'train'])
            # dev:1108 samples , test:2610 samples , train:9989 samples
            # print(len(output["dev"]),len(output["test"]),len(output["train"]))
            # print(output["dev"])
        except yaml.YAMLError as exc:
            print(exc)
    for file in csv_files_train:
        for dtt in ['dev', 'test', 'train']:
            if file in output[dtt]:
                x, y=file, output[dtt][file]["Emotion"]
                data_dict[x] = one_hot(label_transform[y])
                break
    tmp_feature = {}
    for csv_file in csv_files_train:
        data = pd.read_csv(csv_path_train + '/'+ csv_file + '.csv')
        feature = np.array(data, dtype= "float32")[:,2:].tolist()
        tmp_feature[csv_file]=feature
    # audio_feature = torch.tensor(tmp_feature)  # (n,num_time_step,39)
    # print(audio_feature.shape)
    # #print(audio_feature)
    # print(csv_files_train[:15])
    return data_dict, tmp_feature

In [244]:
xy_train=get_data_train()
print(type(xy_train))

<class 'tuple'>


In [65]:
print(xy_train)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [245]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [246]:

class Mydata_train(Dataset):
    def __init__(self):
        super().__init__()
        self.xy_train=xy_train  # a dict
    def __getitem__(self, index):
        file_name_train=list(self.xy_train[0].keys())[index]
        # data, emotion pair
        #nor = (torch.Tensor(self.xy_train[1][file_name_train])-torch.mean(torch.Tensor(self.xy_train[1][file_name_train]),dim=1,keepdim=True))/torch.std(torch.Tensor(self.xy_train[1][file_name_train]),dim=1,keepdim=True)
        nor = torch.Tensor(self.xy_train[1][file_name_train])
        sample_train = nor, torch.Tensor(self.xy_train[0][file_name_train])
        return sample_train
    def __len__(self):
        return len(self.xy_train[0])


import random
def collate_batch(batch):
    label_list, text_list, = [], []
    for (_text, _label) in batch:
        label_list.append(_label.tolist())
        text_list.append(_text)
    #print(label_list)
    label_list = torch.Tensor(label_list)
    text_list = torch.nn.utils.rnn.pad_sequence(text_list, batch_first=True, padding_value=0)
    return text_list, label_list

In [247]:
# Hyper-parameters
num_classes = 7
num_epochs = 2
batch_size = 1
learning_rate = 0.001

input_size = 39 #dim feature
sequence_length = 28
hidden_size = 128
num_layers = 1

In [248]:
d_train=Mydata_train()
print(len(d_train))

# Data loader
train_loader = DataLoader(dataset=d_train,batch_size=batch_size,collate_fn=collate_batch,shuffle=True)

14


In [249]:
for i, (mfccdata, labels) in enumerate(train_loader):
    # origin shape: [N, NN, 39]
    # mfccdata = mfccdata.reshape(-1, sequence_length, input_size).to(device)
    mfccdata = mfccdata.to(device)
    labels = torch.argmax(labels,dim=1)
    print(mfccdata, labels)
    if i == 2:
        break

    # Forward pass
    #outputs = model(mfccdata)
    #loss = criterion(outputs, labels)

tensor([[[-1.3030e+01,  3.5267e+00, -8.2260e+00,  ..., -9.7702e-01,
           8.6326e-01, -2.7303e-01],
         [-1.3329e+01,  5.2657e+00, -2.7618e+00,  ...,  5.3907e-01,
           7.2698e-01, -2.7886e-01],
         [-1.2651e+01,  2.4219e+00, -1.1064e+01,  ...,  1.2016e+00,
          -7.4175e-02, -1.2714e-01],
         ...,
         [-1.0836e+01,  7.5187e+00,  2.5220e+01,  ..., -2.0832e-01,
          -4.3584e-01, -1.8836e-01],
         [-1.5357e+01,  9.4457e+00,  2.7792e+01,  ...,  2.8587e-01,
          -2.4627e-03,  1.9659e-01],
         [-2.1884e+01,  1.0740e+01,  3.2287e+01,  ...,  5.5600e-01,
           7.7045e-01,  3.8818e-01]]], device='cuda:0') tensor([0])
tensor([[[-16.9995,   0.5699,  10.3545,  ...,   1.2961,   1.7076,  -0.3927],
         [-18.8424,  -4.9882,   6.5081,  ...,   0.2750,   1.2597,  -0.2478],
         [-18.2444,   2.7000,   2.6667,  ...,  -1.2331,  -0.2474,  -0.0887],
         ...,
         [-13.5553, -17.8618, -17.0174,  ...,   1.0072,   1.0479,  -4.2384],
   

In [250]:
data = iter(train_loader)._next_data()
type(data)
data_dict, tmp_feature = data

In [251]:
print('data_dict',data_dict.size())
print('tmp_feature',tmp_feature.size())

data_dict torch.Size([1, 96, 39])
tmp_feature torch.Size([1, 7])


In [252]:
#model

class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        # -> x needs to be: (batch_size, seq, input_size)
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, bidirectional=False)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Set initial hidden states (and cell states for LSTM)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        # x: (n, 28, 28), h0: (2, n, 128)
        # Forward propagate GRU
        out, _ = self.gru(x, h0)
        # out: tensor of shape (batch_size, seq_length, hidden_size)
        # out: (n, 28, 128)
        # Decode the hidden state of the last time step
        out = out[:, -1, :]
        # out: (n, 128)
        out = self.fc(out)
        # out: (n, 10)
        return out

In [253]:
model = GRU(input_size, hidden_size, num_layers, num_classes)

In [254]:

tmp_feature = []

for i, (mfccdata, labels) in enumerate(train_loader):
    # origin shape: [N, NN, 39]
    # mfccdata = mfccdata.reshape(-1, sequence_length, input_size).to(device)
    mfccdata = mfccdata
    labels = torch.argmax(labels,dim=1)
    #print(mfccdata, labels)
    # Forward pass
    outputs = model(mfccdata)
    outputs = outputs.detach().numpy() 
    print(outputs,type(outputs))
    tmp_feature.append(outputs)
    #tensor.detach.numpy()
    #print(outputs.shape)

[[-0.00597285  0.33486202  0.56011283  0.4655632  -0.06833297 -0.02077667
   0.235173  ]] <class 'numpy.ndarray'>
[[ 0.00293854  0.718094    0.20659319  0.20457831  0.05012473 -0.03791402
   0.12174387]] <class 'numpy.ndarray'>
[[ 0.00413328  0.7338797   0.1916953   0.1578171  -0.232506   -0.09469978
   0.06562211]] <class 'numpy.ndarray'>
[[-0.23676828  0.4913298   0.43116328  0.37239733 -0.16658399 -0.16470546
   0.2593767 ]] <class 'numpy.ndarray'>
[[-0.20504594  0.7021653  -0.05832031  0.5392014  -0.312952    0.00070646
   0.31331074]] <class 'numpy.ndarray'>
[[ 0.13387406  0.7309747   0.32785875  0.12086102 -0.38612565  0.04860728
   0.24248596]] <class 'numpy.ndarray'>
[[-0.17883302  0.54422474  0.22406659  0.17864805 -0.34631732  0.22685272
   0.21748374]] <class 'numpy.ndarray'>
[[-0.07782893  0.39946008  0.92588353  0.22288829 -0.15131548 -0.00866509
   0.2967993 ]] <class 'numpy.ndarray'>
[[-0.05144903  0.7752793   0.01687102  0.32677487 -0.07162222 -0.32575542
   0.29925823]

In [162]:
print(tmp_feature)
print(torch.tensor(tmp_feature).shape)

[array([[-0.25667688,  0.7707814 , -0.09880745, -0.61482984, -0.8358287 ,
         0.31941494, -0.40392566]], dtype=float32), array([[-0.13106161,  0.8633822 , -0.11533121, -0.06397115, -0.6422912 ,
         0.13768476, -0.63507354]], dtype=float32), array([[-0.2883257 ,  0.7432375 ,  0.0088351 , -0.17591308, -0.8168704 ,
         0.4284621 , -0.1895282 ]], dtype=float32), array([[-0.21261677,  0.7016494 ,  0.02871137,  0.03945391, -0.4810274 ,
         0.24851874, -0.24236003]], dtype=float32), array([[-0.42240456,  0.52366865,  0.01182967, -0.3933382 , -0.6548294 ,
         0.0786937 , -0.5017521 ]], dtype=float32), array([[-0.15000695,  0.8138542 ,  0.04334982,  0.01719181, -0.8008332 ,
         0.21447778, -0.1260587 ]], dtype=float32), array([[-0.4591094 ,  0.61273706,  0.08432996, -0.11785401, -0.8977099 ,
         0.23411539, -0.24712068]], dtype=float32), array([[-0.21843132,  0.5437145 ,  0.3142205 , -0.19169141, -0.90294546,
         0.19101995, -0.31469637]], dtype=float32),

In [168]:
audio_feature = torch.tensor(tmp_feature).unsqueeze(1).reshape(14,-1,7)
print(audio_feature.shape,'\n', audio_feature)

torch.Size([14, 1, 7]) 
 tensor([[[-0.2567,  0.7708, -0.0988, -0.6148, -0.8358,  0.3194, -0.4039]],

        [[-0.1311,  0.8634, -0.1153, -0.0640, -0.6423,  0.1377, -0.6351]],

        [[-0.2883,  0.7432,  0.0088, -0.1759, -0.8169,  0.4285, -0.1895]],

        [[-0.2126,  0.7016,  0.0287,  0.0395, -0.4810,  0.2485, -0.2424]],

        [[-0.4224,  0.5237,  0.0118, -0.3933, -0.6548,  0.0787, -0.5018]],

        [[-0.1500,  0.8139,  0.0433,  0.0172, -0.8008,  0.2145, -0.1261]],

        [[-0.4591,  0.6127,  0.0843, -0.1179, -0.8977,  0.2341, -0.2471]],

        [[-0.2184,  0.5437,  0.3142, -0.1917, -0.9029,  0.1910, -0.3147]],

        [[-0.1743,  0.5670, -0.2766, -0.6428, -0.5938,  0.0883, -0.4031]],

        [[-0.2770,  0.7037, -0.0421,  0.0627, -0.6169,  0.2186, -0.5402]],

        [[-0.4071,  0.3374,  0.0490, -0.4913, -0.7710,  0.4132, -0.4509]],

        [[-0.2545,  0.8028, -0.0142, -0.0514, -0.6649,  0.4576, -0.2078]],

        [[-0.1610,  0.5309,  0.3699, -0.0494, -0.6935,  0.2832,

In [ ]:

#combine models 

In [169]:
import sys
sys.path.append('/content/drive/MyDrive/COSMIC/erc-training')
from dataloader import MELDRobertaCometDataset
from torch.utils.data import DataLoader

In [170]:
classify = 'emotion'

In [173]:
!pwd

'pwd' is not recognized as an internal or external command,
operable program or batch file.


In [175]:
trainset = MELDRobertaCometDataset('train', classify)
train_loader = DataLoader(trainset,
                            batch_size=1,
                            collate_fn=trainset.collate_fn,
                            num_workers=0,
                            pin_memory=False)

In [176]:
data = iter(train_loader)._next_data()

In [177]:
type(data), len(data)

(list, 17)

In [178]:
r1, r2, r3, r4, \
x1, x2, x3, x4, x5, x6, \
o1, o2, o3, \
qmask, umask, label = data[:-1]

In [179]:
[x.size() for x in [r1, r2, r3, r4]]  # (seq_len, batch, hidden_size)


[torch.Size([14, 1, 1024]),
 torch.Size([14, 1, 1024]),
 torch.Size([14, 1, 1024]),
 torch.Size([14, 1, 1024])]

In [180]:
[x.size() for x in [x1, x2, x3, x4, x5, x6]]  # (seq_len, batch, dim_size)


[torch.Size([14, 1, 768]),
 torch.Size([14, 1, 768]),
 torch.Size([14, 1, 768]),
 torch.Size([14, 1, 768]),
 torch.Size([14, 1, 768]),
 torch.Size([14, 1, 768])]

In [181]:
[x.size() for x in [o1, o2, o3]]  # (seq_len, batch, hidden_size)

[torch.Size([14, 1, 768]), torch.Size([14, 1, 768]), torch.Size([14, 1, 768])]

In [182]:
# test
r = (r1 + r2 + r3 + r4) / 4.0
mixed_feature = torch.cat((r, audio_feature), dim=-1)

r.size(), mixed_feature.size()

(torch.Size([14, 1, 1024]), torch.Size([14, 1, 1031]))

In [ ]:
#modify models

In [183]:
from torch import nn
import torch.nn.functional as F
from commonsense_model import CommonsenseGRUModel

In [186]:
class CommonsenseGRUModel_bimodal(CommonsenseGRUModel):

    def __init__(self, 
        D_m, D_s, D_g, D_p, D_r, D_i, D_e, D_h, D_a=100, D_audio=7, 
        n_classes=7, listener_state=False, context_attention='simple', 
        dropout_rec=0.5, dropout=0.1, emo_gru=True, mode1=0, norm=0, residual=False
        ):
        # 增加了一个新的参数 D_audio=384，audio feature 的维度

        super().__init__(
            D_m, D_s, D_g, D_p, D_r, D_i, D_e, D_h, D_a=D_a, 
            n_classes=n_classes, listener_state=listener_state, context_attention=context_attention, 
            dropout_rec=dropout_rec, dropout=dropout, emo_gru=emo_gru, mode1=mode1, norm=norm, residual=residual)

        if mode1 == 0:
            D_x = 4 * D_m
        elif mode1 == 1:
            D_x = 2 * D_m
        else:
            D_x = D_m

        self.linear_in_mixed_feature = nn.Linear(D_x + D_audio, D_h)  # D_h 将传给 CommonsenseRNNCell 作为 D_m，即 dim of mixed utterance feature

    def forward(self, 
                r1, r2, r3, r4, audio_feature, 
                x1, x2, x3, o1, o2, 
                qmask, umask, att2=False, return_hidden=False
                ):
        # 增加了 audio_feature 作为语音特征接口
        """
        U -> seq_len, batch, D_m
        qmask -> seq_len, batch, party
        """

        seq_len, batch, feature_dim = r1.size()

        # args.norm default is 0, namely not normalizing r1 - r4
        if self.norm_strategy == 1:
            r1 = self.norm1a(r1.transpose(0, 1).reshape(-1, feature_dim)).reshape(-1, seq_len, feature_dim).transpose(1, 0)
            r2 = self.norm1b(r2.transpose(0, 1).reshape(-1, feature_dim)).reshape(-1, seq_len, feature_dim).transpose(1, 0)
            r3 = self.norm1c(r3.transpose(0, 1).reshape(-1, feature_dim)).reshape(-1, seq_len, feature_dim).transpose(1, 0)
            r4 = self.norm1d(r4.transpose(0, 1).reshape(-1, feature_dim)).reshape(-1, seq_len, feature_dim).transpose(1, 0)

        elif self.norm_strategy == 2:
            norm2 = nn.LayerNorm((seq_len, feature_dim), elementwise_affine=False)
            r1 = norm2(r1.transpose(0, 1)).transpose(0, 1)
            r2 = norm2(r2.transpose(0, 1)).transpose(0, 1)
            r3 = norm2(r3.transpose(0, 1)).transpose(0, 1)
            r4 = norm2(r4.transpose(0, 1)).transpose(0, 1)

        elif self.norm_strategy == 3:
            r1 = self.norm3a(r1.transpose(0, 1).reshape(-1, feature_dim)).reshape(-1, seq_len, feature_dim).transpose(1, 0)
            r2 = self.norm3b(r2.transpose(0, 1).reshape(-1, feature_dim)).reshape(-1, seq_len, feature_dim).transpose(1, 0)
            r3 = self.norm3c(r3.transpose(0, 1).reshape(-1, feature_dim)).reshape(-1, seq_len, feature_dim).transpose(1, 0)
            r4 = self.norm3d(r4.transpose(0, 1).reshape(-1, feature_dim)).reshape(-1, seq_len, feature_dim).transpose(1, 0)

        if self.mode1 == 0:
            r = torch.cat([r1, r2, r3, r4], axis=-1)
        elif self.mode1 == 1:
            r = torch.cat([r1, r2], axis=-1)
        elif self.mode1 == 2:
            r = (r1 + r2 + r3 + r4)/4
        elif self.mode1 == 3:
            r = r1
        elif self.mode1 == 4:
            r = r2
        elif self.mode1 == 5:
            r = r3
        elif self.mode1 == 6:
            r = r4
        elif self.mode1 == 7:
            r = self.r_weights[0]*r1 + self.r_weights[1]*r2 + self.r_weights[2]*r3 + self.r_weights[3]*r4
        
        # combination roberta feature with audio feature
        # r [seq_len, batch_size, dim_roberta]
        # audio_feature [seq_len, batch_size, dim_audio]
        mixed_feature = torch.cat((r, audio_feature), dim=-1)  # [seq_len, batch_size, dim_mixed_feature]
        r = self.linear_in_mixed_feature(mixed_feature)
        
        emotions_f, alpha_f = self.cs_rnn_f(r, x1, x2, x3, o1, o2, qmask)     
        out_sense, _ = self.sense_gru(x1)
        
        rev_r = self._reverse_seq(r, umask)
        rev_x1 = self._reverse_seq(x1, umask)
        rev_x2 = self._reverse_seq(x2, umask)
        rev_x3 = self._reverse_seq(x3, umask)
        rev_o1 = self._reverse_seq(o1, umask)
        rev_o2 = self._reverse_seq(o2, umask)
        rev_qmask = self._reverse_seq(qmask, umask)
        emotions_b, alpha_b = self.cs_rnn_r(rev_r, rev_x1, rev_x2, rev_x3, rev_o1, rev_o2, rev_qmask)
        emotions_b = self._reverse_seq(emotions_b, umask)
        
        emotions = torch.cat([emotions_f,emotions_b],dim=-1)
        emotions = self.dropout_rec(emotions)
        
        alpha, alpha_f, alpha_b = [], [], []
        if att2:
            att_emotions = []
            alpha = []
            for t in emotions:
                att_em, alpha_ = self.matchatt(emotions,t,mask=umask)
                att_emotions.append(att_em.unsqueeze(0))
                alpha.append(alpha_[:,0,:])
            att_emotions = torch.cat(att_emotions,dim=0)
            hidden = F.relu(self.linear(att_emotions))
        else:
            hidden = F.relu(self.linear(emotions))
            
        hidden = self.dropout(hidden)
        
        if self.residual:
            hidden = hidden + r
        
        log_prob = F.log_softmax(self.smax_fc(hidden), 2)

        if return_hidden:
            return hidden, alpha, alpha_f, alpha_b, emotions
        return log_prob, out_sense, alpha, alpha_f, alpha_b, emotions

In [187]:
seed = 1

no_cuda = True # 是否禁用 cuda
cuda = torch.cuda.is_available() and not no_cuda

emo_gru = True
batch_size = 1
classify = 'emotion'
n_classes = 7 if classify == 'emotion' else 3 if classify == 'sentiment' else 0

print('Running %s classification task on %s' % (classify, 'GPU' if cuda else 'CPU')) 

Running emotion classification task on CPU


In [200]:
D_m = 1024  # roberta 的四个向量的维度，这四个向量平均之后取自 [CLS]在最后四层 hidden representations，四个向量平均之后作为 utterance 的表示
D_s = 768  # 5种 常识向量 的 维度 (commonsense)
D_g = 150  # dim of context state (global)

D_p = 150  # dim of internal state
D_r = 150  # dim of external state
D_i = 150  # dim of intet state

D_h = 100  # 将输入从 D_m 变换到 D_h，模型内部的维度
D_a = 100  # dim of attention
D_audio = 7

D_e = D_p + D_r + D_i

In [201]:
model = CommonsenseGRUModel_bimodal(
    D_m, D_s, D_g, D_p, D_r, D_i, D_e, D_h, D_a=D_a, D_audio=D_audio, 
    n_classes=n_classes, listener_state=True, context_attention='simple',
    dropout_rec=0.5, dropout=0.5, emo_gru=True, mode1=2, norm=0, residual=True)
if cuda:
    model.cuda()

In [ ]:
#forward

In [202]:
import random

In [203]:
model.eval()

# set seed
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [204]:
# forward 
log_prob, out_sense, alpha, alpha_f, alpha_b, emotions = model(r1, r2, r3, r4, audio_feature, x5, x6, x1, o2, o3, qmask, umask, att2=False)

In [205]:
# shou results
[type(x) for x in [log_prob, out_sense, alpha, alpha_f, alpha_b, emotions]]

[torch.Tensor, torch.Tensor, list, list, list, torch.Tensor]

In [206]:
[x.size() for x in [log_prob, out_sense, emotions]]
# log_prob   [14, 1, 7]    (seq_len, batch, num_emotions)
# 七种 emotion 的预测概率

# out_sense  [14, 1, 768]  (seq_len, batch, hidden_size)
# self.sense_gru = nn.GRU(input_size=D_s, hidden_size=D_s//2, num_layers=1, bidirectional=True)
# out_sense, _ = self.sense_gru(x1)
# 不太明白 out_sense 是什么

[torch.Size([14, 1, 7]), torch.Size([14, 1, 768]), torch.Size([14, 1, 900])]

In [207]:
log_prob

tensor([[[-2.3965, -1.7546, -2.5307, -1.9847, -2.0252, -2.2455, -1.2689]],

        [[-2.2774, -1.6777, -2.4191, -1.9318, -1.9595, -2.3078, -1.4425]],

        [[-2.3356, -1.8608, -2.3524, -2.1180, -2.0980, -2.0582, -1.2661]],

        [[-2.5710, -1.7643, -2.1608, -2.0205, -2.3415, -2.0340, -1.2822]],

        [[-2.6927, -1.6970, -2.2761, -2.1009, -1.9304, -2.0086, -1.4074]],

        [[-2.3415, -1.8356, -2.3316, -2.0417, -1.9331, -2.2447, -1.3217]],

        [[-2.3137, -1.8975, -2.2358, -1.9741, -2.0540, -1.8869, -1.4889]],

        [[-2.1993, -1.8003, -2.2926, -1.8849, -1.9544, -2.3445, -1.4546]],

        [[-2.3081, -1.9500, -2.2053, -1.9561, -1.9704, -2.0817, -1.4167]],

        [[-2.4409, -2.0596, -2.2035, -2.0172, -2.1868, -2.0494, -1.2010]],

        [[-2.1670, -1.9257, -2.1600, -1.8062, -2.1574, -2.3304, -1.3975]],

        [[-2.2026, -1.9458, -2.4334, -1.7872, -2.1213, -2.1109, -1.3845]],

        [[-2.5038, -1.8859, -2.2161, -1.7887, -2.0049, -2.2290, -1.3943]],

        [[-2

tutorials combine models 
1.https://github.com/Min-Sheng/multimodal-speech-emotion-recognizer/blob/master/model/multi_modal_model.py
2.https://github.com/Lev-etd/Multimodal-emotion-recognition/blob/master/Model.ipynb
3.https://github.com/mishra-pa/Multimodal-Transformer-for-Emotion-Recognition/blob/main/src/models.py
4.https://github.com/PoYuYang/EE599-Multi-modal-Sentiment-Analysis-Project
5.https://github.com/HandsomeQD/MVAE_Bi-LSTMs_VGG19_pytorch/blob/master/BiLSTM_VGG_VAE.ipynb
6.https://github.com/yuanxiaosc/Multimodal-short-video-dataset-and-baseline-classification-model/blob/master/baseline_model/mutimodal_baseline_model.py
7.https://github.com/bkoch4142/multimodal-emotion-classification/blob/main/src/models/multimodal_model.py
8.https://github.com/mebasiri/Multimodal-Persian-Sentiment-Analysis/blob/main/Multimodal_Persian_SA.ipynb
9.https://github.com/sayakpaul/Multimodal-Entailment-Baseline
10.https://github.com/david-yoon/multimodal-speech-emotion/blob/master/preprocessing/IEMOCAP_01_wav_to_feature.ipynb
11.https://github.com/aris-ai/Audio-and-text-based-emotion-recognition/blob/master/Text_and_Audio_Emotion.ipynb
12.https://github.com/TianyiWu96/Multimodal-Sentiment/blob/master/multimodel.ipynb

In [ ]:
#audio GRN  models

In [45]:
class AUDIOGRU(nn.Module):
    def __init__(self, ainput_size, ahidden_size, anum_layers, anum_classes):
        super().__init__()
        self.anum_layers = anum_layers
        self.ahidden_size = ahidden_size
        # -> x needs to be: (batch_size, seq, input_size)
        self.agru = nn.GRU(ainput_size, ahidden_size, anum_layers, batch_first=True, bidirectional=False)
        self.afc = nn.Linear(ahidden_size, anum_classes)

    def forward(self, ax):
        # Set initial hidden states (and cell states for LSTM)
        h0 = torch.zeros(self.anum_layers, ax.size(0), self.ahidden_size)
        # x: (n, 28, 28), h0: (2, n, 128)
        # Forward propagate GRU
        out, _ = self.agru(ax, h0)
        # out: tensor of shape (batch_size, seq_length, hidden_size)
        # out: (n, 28, 128)
        # Decode the hidden state of the last time step
        out = out[:, -1, :]
        # out: (n, 128)
        out = self.afc(out)
        # out: (n, 10)
        return out

In [46]:
# Hyper-parameters
#AudioGRU
ainput_size = 39 
ahidden_size = 128
anum_layers = 1
anum_classes = 7

#COSMICGRU
cinput_size = 39 
chidden_size = 128
cnum_layers = 1
cnum_classes = 7

#linear
output_dim = 60
    
model = AUDIOGRU(ainput_size, ahidden_size, anum_layers, anum_classes)
print(model)


AUDIOGRU(
  (agru): GRU(39, 128, batch_first=True)
  (afc): Linear(in_features=128, out_features=7, bias=True)
)


In [28]:
#params layers change 
params = model.named_parameters()
for name, param in params:
    print(name, param.data.shape)
    print(type(param))

agru.weight_ih_l0 torch.Size([384, 39])
<class 'torch.nn.parameter.Parameter'>
agru.weight_hh_l0 torch.Size([384, 128])
<class 'torch.nn.parameter.Parameter'>
agru.bias_ih_l0 torch.Size([384])
<class 'torch.nn.parameter.Parameter'>
agru.bias_hh_l0 torch.Size([384])
<class 'torch.nn.parameter.Parameter'>
afc.weight torch.Size([7, 128])
<class 'torch.nn.parameter.Parameter'>
afc.bias torch.Size([7])
<class 'torch.nn.parameter.Parameter'>


In [26]:
input = torch.randn(5, 3, 39)
model(input)

tensor([[ 0.1446,  0.0278,  0.0085, -0.0728, -0.2254, -0.0280, -0.1586],
        [ 0.0435,  0.0471, -0.0866, -0.0736, -0.0048,  0.1223,  0.0235],
        [ 0.1835, -0.0421, -0.2117, -0.1400,  0.1453,  0.1927,  0.0646],
        [-0.0168, -0.2436,  0.0993, -0.1072, -0.0432,  0.1045, -0.0435],
        [ 0.0899,  0.1206,  0.0256, -0.0424, -0.1652,  0.1340,  0.0894]],
       grad_fn=<AddmmBackward>)

In [47]:
from torchsummary import summary

#notice h0必须cuda设备 model必须cuda设备
model = model.cuda()
summary(model, (3, 39))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
               GRU-1  [[-1, 3, 128], [-1, 2, 128]]               0
            Linear-2                    [-1, 7]             903
Total params: 903
Trainable params: 903
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.75
Params size (MB): 0.00
Estimated Total Size (MB): 0.75
----------------------------------------------------------------


In [40]:
from torchvision import models
from torchsummary import summary


resnet18 = models.resnet18().cuda()
summary(resnet18, (3, 224, 224))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

#audio cosmic models 

In [226]:
class COSMICGRU(nn.Module):
    def __init__(self, cinput_size, chidden_size, cnum_layers, cnum_classes):
        super().__init__()
        self.cnum_layers = cnum_layers
        self.chidden_size = chidden_size
        # -> x needs to be: (batch_size, seq, input_size)
        self.cgru = nn.GRU(cinput_size, chidden_size, cnum_layers, batch_first=True, bidirectional=False)
        self.cfc = nn.Linear(chidden_size, cnum_classes)

    def forward(self, cx):
        # Set initial hidden states (and cell states for LSTM)
        h0 = torch.zeros(self.cnum_layers, cx.size(0), self.chidden_size)
        # x: (n, 28, 28), h0: (2, n, 128)
        # Forward propagate GRU
        out, _ = self.cgru(cx, h0)
        # out: tensor of shape (batch_size, seq_length, hidden_size)
        # out: (n, 28, 128)
        # Decode the hidden state of the last time step
        out = out[:, -1, :]
        # out: (n, 128)
        out = self.cfc(out)
        # out: (n, 10)
        return out

In [235]:
COSMICGRU(cinput_size,chidden_size, cnum_layers, cnum_classes)

COSMICGRU(
  (cgru): GRU(39, 128, batch_first=True)
  (cfc): Linear(in_features=128, out_features=7, bias=True)
)

#combine models 

In [269]:
class MultimodalModal(nn.Module):
    
    def __init__(self,ainput_size, ahidden_size, anum_layers, anum_classes, cinput_size, chidden_size, cnum_layers, cnum_classes):
        super(MultimodalModal,self).__init__()
        self.ainput_size = ainput_size
        self.ahidden_size = ahidden_size
        self.anum_layers = anum_layers
        self.anum_classes = anum_classes
        self.cinput_size = cinput_size
        self.chidden_size = chidden_size
        self.cnum_layers = cnum_layers
        self.cnum_classes = cnum_classes
        self.output_dim = output_dim
        
        self.AUDIOGRU = AUDIOGRU(ainput_size = self.ainput_size, ahidden_size = self.ahidden_size, anum_layers = self.anum_layers, anum_classes = self.anum_classes)
        
        self.COSMICGRU = COSMICGRU(cinput_size = self.cinput_size,chidden_size = self.chidden_size, cnum_layers = self.cnum_layers, cnum_classes =  self.cnum_classes)
    
        #self.out = nn.Linear(int(self.ahidden_size + self.chidden_size),self.output_dim)
        
    def forward(self, ax, cx):
        
        out_audioGRU = self.AUDIOGRU(ax) 
        out_audioCosmicGRU = self.COSMICGRU(cx)
        final = torch.cat((out_audioGRU, out_audioCosmicGRU), dim=1)
        print(final.shape)
        return final   

In [271]:
MultimodalModal = MultimodalModal(ainput_size, ahidden_size, anum_layers, anum_classes, cinput_size, chidden_size, cnum_layers, cnum_classes)

In [60]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from combine_models.single_text_model import COSMICGRU
from combine_models.single_audio_model import AUDIOGRU

class CMultimodalModal(nn.Module):
    
    def __init__(self,ainput_size, ahidden_size, anum_layers, anum_classes, cinput_size, chidden_size, cnum_layers, cnum_classes):
        super(CMultimodalModal,self).__init__()
        self.ainput_size = ainput_size
        self.ahidden_size = ahidden_size
        self.anum_layers = anum_layers
        self.anum_classes = anum_classes
        self.cinput_size = cinput_size
        self.chidden_size = chidden_size
        self.cnum_layers = cnum_layers
        self.cnum_classes = cnum_classes
        self.output_dim = output_dim
        
        self.AUDIOGRU = AUDIOGRU(ainput_size = self.ainput_size, ahidden_size = self.ahidden_size, anum_layers = self.anum_layers, anum_classes = self.anum_classes)
        
        self.COSMICGRU = COSMICGRU(cinput_size = self.cinput_size,chidden_size = self.chidden_size, cnum_layers = self.cnum_layers, cnum_classes =  self.cnum_classes)
    
        #self.out = nn.Linear(int(self.ahidden_size + self.chidden_size),self.output_dim)
        
    def forward(self, ax, cx):
        
        out_audioGRU = self.AUDIOGRU(ax) 
        out_audioCosmicGRU = self.COSMICGRU(cx)
        final = torch.cat((out_audioGRU, out_audioCosmicGRU), dim=1)
        print(final.shape)
        return final   


In [61]:
CMultimodalModal = CMultimodalModal(ainput_size, ahidden_size, anum_layers, anum_classes, cinput_size, chidden_size, cnum_layers, cnum_classes)

In [55]:
# Hyper-parameters
#AudioGRU
ainput_size = 39 
ahidden_size = 128
anum_layers = 1
anum_classes = 7

#COSMICGRU
cinput_size = 39 
chidden_size = 128
cnum_layers = 1
cnum_classes = 7

#linear
output_dim = 60

In [ ]:
import numpy as np, argparse 


In [272]:
combine_feature = []

for i, (mfccdata, labels) in enumerate(train_loader):
    # origin shape: [N, NN, 39]
    # mfccdata = mfccdata.reshape(-1, sequence_length, input_size).to(device)
    mfccdata = mfccdata
    labels = torch.argmax(labels,dim=1)
    #print(mfccdata, labels)
    # Forward pass
    outputs = MultimodalModal(mfccdata,mfccdata)
    print(outputs.shape)
    outputs = outputs.detach().numpy() 
    print(outputs,type(outputs))
    combine_feature.append(outputs)
    #tensor.detach.numpy()
    #print(outputs.shape)

torch.Size([1, 14])
torch.Size([1, 14])
[[ 0.6812245  -0.04862917 -0.10408711 -0.10611243 -0.09928183 -0.6035498
  -0.2471769   0.24714881 -0.49965137 -0.71538013 -0.3036044   0.16739881
  -0.06476932  0.37533092]] <class 'numpy.ndarray'>
torch.Size([1, 14])
torch.Size([1, 14])
[[ 0.6581431  -0.737012   -0.07999098 -0.19144666 -0.12357993 -0.5105235
  -0.32763776  0.61912507 -0.31017584 -0.78109425 -0.6432464  -0.03616352
  -0.44865024  0.5261647 ]] <class 'numpy.ndarray'>
torch.Size([1, 14])
torch.Size([1, 14])
[[ 0.56383485 -0.5124508   0.06158877 -0.2273463  -0.54517084 -0.4390095
  -0.28686917  0.5619225   0.16593805 -0.88103575 -0.3937074  -0.05868623
  -0.4033925   0.18011972]] <class 'numpy.ndarray'>
torch.Size([1, 14])
torch.Size([1, 14])
[[ 0.90701854 -0.3746734  -0.15673012 -0.09624866 -0.04575521 -0.6883424
  -0.33255118  0.21348864 -0.42830467 -0.7697725  -0.54777426 -0.059483
  -0.50974697  0.32665145]] <class 'numpy.ndarray'>
torch.Size([1, 14])
torch.Size([1, 14])
[[ 0.5

# attention tutorials

attention

'''
动手深度学习Page 407 and Page 410

'''

In [8]:
import torch
from torch import nn
import math

In [319]:

class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        # -> x needs to be: (batch_size, seq, input_size)
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, bidirectional=False)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Set initial hidden states (and cell states for LSTM)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        # x: (n, 28, 28), h0: (2, n, 128)
        # Forward propagate GRU
        out, _ = self.gru(x, h0)
        # out: tensor of shape (batch_size, seq_length, hidden_size)
        # out: (n, 28, 128)
        # Decode the hidden state of the last time step
        out = out[:, -1, :]
        # out: (n, 128)
        out = self.fc(out)
        # out: (n, 10)
        return out

In [9]:
class DotProductAttention(nn.Module):
    def __init__(self, dropout, **kwargs):
        super(DotProductAttention, self).__init__(**kwargs)
        self.dropout = nn.Dropout(dropout)
        # Shape of `queries`: (`batch_size`, no. of queries, `d`)
        # Shape of `keys`: (`batch_size`, no. of key-value pairs, `d`)
        # Shape of `values`: (`batch_size`, no. of key-value pairs, value
        # dimension)
        # Shape of `valid_lens`: (`batch_size`,) or (`batch_size`, no. of queries)
        
    def forward(self, queries, keys, values):
        d = queries.shape[-1]
        # Set `transpose_b=True` to swap the last two dimensions of `keys`
        scores = torch.bmm(queries, keys.transpose(1,2)) / math.sqrt(d)
        return torch.bmm(self.dropout(scores), values)

In [286]:
'''
The scaled dot-product attention of queries Q ∈ R n×d, keys K ∈ R m×d, and values V ∈ R m×v  

Encoder  <<< multi-head self attention 动手深度学习Page426

'''
queries = torch.normal(0, 1, (2, 1, 2))
keys = torch.normal(0, 1, (2, 1, 2))
values = torch.normal(0, 1, (2, 1, 2))

attention = DotProductAttention(dropout=0.5)
attention.eval()
attention = attention(queries, keys, values)


In [290]:
print(attention.shape)
print(type(attention))


torch.Size([2, 1, 2])
<class 'torch.Tensor'>


In [291]:
attention.reshape(1,1,2,2)

tensor([[[[-0.8423,  0.0613],
          [-0.2232, -0.0457]]]])

Multihead attention

In [297]:
def transpose_qkv(X, num_heads):
    X = X.reshape(X.shape[0], X.shape[1], num_heads, -1)
    X = X.permute(0, 2, 1, 3)
    return X.reshape(-1, X.shape[2], X.shape[3])

def transpose_output(X, num_heads): 
    X = X.reshape(-1, num_heads, X.shape[1], X.shape[2]) 
    X = X.permute(0, 2, 1, 3) 
    return X.reshape(X.shape[0], X.shape[1], -1)


In [314]:
class MultiHeadAttention(nn.Module):

    def __init__(self, key_size, query_size, value_size, num_hiddens,
    num_heads, dropout, bias=False, **kwargs):
        super(MultiHeadAttention, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.attention = DotProductAttention(dropout)
        self.W_q = nn.Linear(query_size, num_hiddens, bias=bias)
        self.W_k = nn.Linear(key_size, num_hiddens, bias=bias)
        self.W_v = nn.Linear(value_size, num_hiddens, bias=bias)
        self.W_o = nn.Linear(num_hiddens, num_hiddens, bias=bias)
        
    def forward(self, queries, keys, values):
        # Shape of `queries`, `keys`, or `values`:
        # (`batch_size`, no. of queries or key-value pairs, `num_hiddens`)
        # Shape of `valid_lens`:
        # (`batch_size`,) or (`batch_size`, no. of queries)
        # After transposing, shape of output `queries`, `keys`, or `values`:
        # (`batch_size` * `num_heads`, no. of queries or key-value pairs,
        # `num_hiddens` / `num_heads`)
        queries = transpose_qkv(self.W_q(queries), self.num_heads)
        keys = transpose_qkv(self.W_k(keys), self.num_heads)
        values = transpose_qkv(self.W_v(values), self.num_heads)
     
        output = self.attention(queries, keys, values)
        # Shape of `output_concat`:
        # (`batch_size`, no. of queries, `num_hiddens`)
        output_concat = transpose_output(output, self.num_heads)
        return self.W_o(output_concat)


In [315]:
num_hiddens, num_heads = 100, 5
attention = MultiHeadAttention(num_hiddens, num_hiddens, num_hiddens,num_hiddens, num_heads, 0.5)
attention.eval()


MultiHeadAttention(
  (attention): DotProductAttention(
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (W_q): Linear(in_features=100, out_features=100, bias=False)
  (W_k): Linear(in_features=100, out_features=100, bias=False)
  (W_v): Linear(in_features=100, out_features=100, bias=False)
  (W_o): Linear(in_features=100, out_features=100, bias=False)
)

In [316]:
print(attention)

MultiHeadAttention(
  (attention): DotProductAttention(
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (W_q): Linear(in_features=100, out_features=100, bias=False)
  (W_k): Linear(in_features=100, out_features=100, bias=False)
  (W_v): Linear(in_features=100, out_features=100, bias=False)
  (W_o): Linear(in_features=100, out_features=100, bias=False)
)


In [318]:
batch_size, num_queries, num_kvpairs  = 2, 4, 6
X = torch.ones((batch_size, num_queries, num_hiddens))
Y = torch.ones((batch_size, num_kvpairs, num_hiddens))
attention = attention(X,Y,Y)
print(attention.shape)
print('/n',attention)

torch.Size([2, 4, 100])
/n tensor([[[ 2.2436e-01,  1.7030e-01,  1.5454e-01, -5.4209e-01, -1.1723e+00,
           3.8347e-01,  6.8882e-04,  6.7641e-01, -6.0793e-01, -8.5367e-01,
           1.9934e-01, -8.7626e-01, -3.2894e-01, -8.3591e-01, -1.3181e+00,
          -8.7349e-01, -5.5290e-02, -6.2123e-01, -7.3228e-01, -3.1099e-01,
          -8.2433e-01,  5.2567e-01,  2.9478e-01,  2.6369e-01, -1.0796e-01,
          -1.9514e-02,  2.2952e-01, -7.5005e-01,  5.2535e-01, -1.8827e-01,
          -1.7342e-02, -1.9349e-01, -8.2957e-01,  2.0198e-01,  2.5380e-02,
          -1.4443e+00, -6.6620e-01, -3.4613e-01,  3.2738e-01, -3.4000e-01,
          -4.2898e-01,  4.8691e-01,  2.9637e-01,  9.2918e-01,  3.9288e-02,
          -3.8061e-01,  4.6110e-01,  8.8262e-01, -1.7038e+00, -8.3047e-01,
           9.6022e-02,  2.2452e-01,  1.3397e-01,  1.5740e-01,  3.9711e-01,
           1.7809e+00, -1.4859e-01,  6.1304e-01, -5.1841e-01,  5.9695e-02,
          -6.5050e-01, -5.0063e-02, -2.0790e-02, -9.8684e-02, -8.6913e-01

#Attention GRU Models 

'''
Attention GRU 
经典教程 
1.https://github.com/bkoch4142/multimodal-emotion-classification/blob/main/src/models/multimodal_model.py
2.https://github.com/mebasiri/Multimodal-Persian-Sentiment-Analysis/blob/main/Multimodal_Persian_SA.ipynb
3.https://github.com/TianyiWu96/Multimodal-Sentiment/blob/master/multimodel.ipynb


'''

In [10]:
import argparse
import torch
import torch.nn as nn

'''

input_size = 10
hidden_size = 20
num_layers = 2
dropout = 0.5

'''
class AttentionGRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout,sequence_length,classes, **kwargs ):
        super(AttentionGRU,self).__init__(**kwargs)
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.sequence_length = sequence_length 
        self.num_layers = num_layers
        self.dropout = dropout
        self.classes = classes
        # -> x needs to be: (batch_size, seq, input_size) #rnn = nn.GRU(10, 20, 2)
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, bidirectional=False)
        self.attention =  DotProductAttention(dropout)
        self.fc = nn.Linear(hidden_size*sequence_length,classes)

    
    def forward(self,X):
        h0 = torch.randn(2, 5, 20) #h0 = torch.randn(2, 3, 20)
        output, hn = self.gru(X, h0)
        output = self.attention(output,output,output)
        output = output.view(5, -1)
        output = self.fc(output)
    
        return output  

In [11]:
 #hyper parameters
input_size = 10
hidden_size = 20
num_layers = 2
dropout = 0.5
sequence_length = 3
classes = 1


In [13]:
#train
input = torch.randn(5, 3, 10) #batch_size, sequence length, input_size 
model = AttentionGRU(input_size, hidden_size, num_layers, dropout, sequence_length, classes)
output = attention(input)
print(output.shape)
print('/n',output)


torch.Size([5, 1])
/n tensor([[-0.2684],
        [-0.0181],
        [ 0.2665],
        [ 0.2027],
        [-0.4124]], grad_fn=<AddmmBackward>)


In [25]:
#parse 命令行交互

%run argparseAttentionGRU.py -h


usage: argparseAttentionGRU.py [-h] [-input_size INPUT_SIZE] [-hidden_size HIDDEN_SIZE] [-num_layers NUM_LAYERS]
                               [-dropout DROPOUT]

optional arguments:
  -h, --help            show this help message and exit
  -input_size INPUT_SIZE
                        input_size value
  -hidden_size HIDDEN_SIZE
                        hidden_size value
  -num_layers NUM_LAYERS
                        num_layers value
  -dropout DROPOUT      num_layers value


In [29]:
%run argparseAttentionGRU.py -input_size 10 -hidden_size 20 -num_layers 2 -dropout 0.5


Namespace(dropout=0.5, hidden_size=20, input_size=10, num_layers=2)
torch.Size([5, 3, 20])
/n tensor([[[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 3.4058e-01,  1.6781e-01,  2.1736e-01, -1.8409e-01, -5.5026e-01,
           6.3767e-02, -4.6631e-01, -2.8172e-02,  6.9431e-01,  2.7502e-01,
           2.6080e-01,  8.6338e-01,  2.5791e-01, -1.4861e+00, -4.2175e-02,
          -3.3868e-01,  4.8211e-01, -5.0012e-01, -5.2207e-01, -9.6348e-02],
         [ 1.2039e-01,  5.7068e-02,  1.1424e-01, -8.4617e-02, -2.4293e-01,
           2.0479e-02, -2.3074e-01, -6.6432e-03,  3.2634e-01,  1.1089e-01,
           8.7724e-02,  3.6122e-01,  1.1445e-01, -6.7487e-01, -1.4012e-02,
          -1.7232e-01,  2.2308e-01, -2.1329e-01, -2.1174e-01, -9.5305e-02]],

 

In [33]:
#save models 
torch.save(attention.state_dict(),'ckpt.pt')

In [40]:
#load models
path = "‪C:/Users/faustineljc/ckpt.pt"
path = path.strip('\u202a')

save_file = torch.load(path)
print(type(save_file))
print(save_file)

<class 'collections.OrderedDict'>


In [48]:
for k,v in save_file.items():
    print('parameter: ',k,  'value：', v)

parameter:  gru.weight_ih_l0 value： tensor([[-0.0234,  0.1535,  0.0039, -0.1389,  0.2227, -0.0492, -0.0702, -0.1888,
          0.1331, -0.0688],
        [-0.0030,  0.0479,  0.0667, -0.2131, -0.0623, -0.2042,  0.2223, -0.1043,
         -0.0053,  0.0620],
        [ 0.1338, -0.1320,  0.0929,  0.0018, -0.1536, -0.1907, -0.1617,  0.1501,
          0.1340,  0.0870],
        [ 0.0960,  0.2024, -0.2114,  0.0578,  0.0968, -0.0255,  0.0992, -0.0854,
         -0.1059, -0.1644],
        [-0.0155,  0.0243,  0.1070, -0.0035,  0.1032,  0.0946, -0.0908, -0.1152,
          0.0237,  0.0245],
        [ 0.0461, -0.1165,  0.1645,  0.0752,  0.1066,  0.1112, -0.0887, -0.0754,
         -0.0216,  0.1216],
        [-0.1473, -0.1237, -0.1776,  0.0838,  0.2006, -0.0533,  0.1327, -0.1997,
         -0.2010,  0.1693],
        [-0.2169,  0.1044, -0.1931,  0.0790, -0.0850,  0.0166,  0.2194, -0.0294,
         -0.1334, -0.0756],
        [ 0.1139, -0.1640, -0.1121,  0.1406, -0.0686, -0.1956,  0.2181, -0.1623,
         -0

#nn.Linear models 


#tensorboard 使用tutorials
pytorch数据展示summarywriter的两个函数 
https://blog.csdn.net/M_arshal_/article/details/120324253
https://github.com/christianversloot/machine-learning-articles/blob/main/how-to-use-tensorboard-with-pytorch.md
https://www.jianshu.com/p/46eb3004beca

In [1]:
import torch
import torchvision
from torch.utils.tensorboard import SummaryWriter 
from torchvision import datasets, transforms 

#writer will output to ./runs/directory by default 

writer = SummaryWriter('C:/Users/faustineljc/Desktop')
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,),(0.5,))])
trainset = datasets.MNIST('mnist_train',train = True, download = True, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset,batch_size = 6, shuffle = True)

model = torchvision.models.resnet50(False)
print(model)
model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
images, labels = next(iter(trainloader))

grid = torchvision.utils.make_grid(images)
writer.add_image('images',grid, 0 )
writer.add_graph(model, images)
writer.close()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
!tensorboard --logdir=runs 

In [ ]:
#This can then be visualized wit tensorboard, which should be installed and runnable with 

In [19]:
from torch.utils.tensorboard import SummaryWriter
#log dir:tensorboard 文件的存放路径
writer = SummaryWriter(log_dir = 'runs/exp1')
x = range(100)
for i in x:
    writer.add_scalar('y=2x', i * 2, i)
writer.close()

In [ ]:
#parser 函数
'''
https://www.jianshu.com/p/27ce67dab97e
'''

train models

pytroch dataset dataloader 经典教程

'''
1.https://blog.csdn.net/sdnuwjw/article/details/111227327
2.https://geek-docs.com/pytorch/pytorch-tutorial/pytorch-dataset.html
3.http://www.pytorchmaster.com/5-1%2CDataset%E5%92%8CDataLoader/
'''

In [64]:
#packages load datasets
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

source_data = np.random.randn(100,3, 10)
source_data = torch.from_numpy(source_data)

source_label = np.random.randint(0,6,(100, 1))
source_label =  torch.from_numpy(source_label)
print("#"*len(source_data))


class MyDataSet(Dataset):
    def __init__(self, data_root,data_label):
        self.data = data_root
        self.label= data_label
        
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        data = self.data[idx]
        label = self.label[idx]
        return data, label


#print(type(custom_dataset))

####################################################################################################


In [65]:
custom_dataset = MyDataSet(source_data, source_label)
print(custom_dataset)

In [66]:
from torch.utils.data import DataLoader
def get_loaders(dataset = custom_dataset, batch_size = 5, num_workers  = 0):
 
    return  DataLoader(dataset = dataset, batch_size=batch_size, num_workers = num_workers)

In [67]:
train_loader = get_loaders()
print(train_loader)

In [68]:
for  i, train_sample in enumerate(train_loader):
     print("{} {} ".format(i, train_sample))

0 [tensor([[[ 0.5906,  2.1254, -1.4106, -0.0708, -1.1566,  0.3031,  0.4593,
           0.6527,  0.4387, -0.1382],
         [ 1.0361,  0.8170, -2.2161, -1.2387, -1.0338, -1.1582,  1.0863,
          -1.2731,  0.2998,  1.0527],
         [ 0.9313,  1.5582,  2.1073, -0.7450, -0.0034,  1.5202, -1.3243,
          -0.9366,  2.6180, -0.6339]],

        [[ 0.5543,  0.5932,  0.1466,  0.2104, -0.2425,  0.0335, -0.2221,
           0.9109,  0.3075, -0.5472],
         [ 2.1602, -2.2460,  0.1286,  0.3825,  0.0081,  1.1463,  0.5968,
           3.0428,  0.0080, -2.1197],
         [-0.0711, -2.6423,  1.9394, -0.4242,  0.0202, -2.4865,  1.3386,
           1.6613,  1.0677,  1.8485]],

        [[-0.8536, -1.0123, -0.1331, -0.4215, -0.3352, -0.3252,  0.4758,
           0.3469,  0.3074, -0.3635],
         [ 0.5422,  0.9570, -0.3695, -0.3208,  1.7008, -1.5595,  0.3833,
          -0.9529,  0.9771, -1.1279],
         [ 1.3337, -1.8153, -1.0798,  0.7307, -0.9975, -1.1125,  1.3222,
          -0.7713, -0.8510,  0.7

In [69]:
#train datasets 
train_loader = get_loaders()

for i, data in enumerate(train_loader):
    print("第 {} 个Batch \n{}".format(i, data))
    

第 0 个Batch 
[tensor([[[ 0.5906,  2.1254, -1.4106, -0.0708, -1.1566,  0.3031,  0.4593,
           0.6527,  0.4387, -0.1382],
         [ 1.0361,  0.8170, -2.2161, -1.2387, -1.0338, -1.1582,  1.0863,
          -1.2731,  0.2998,  1.0527],
         [ 0.9313,  1.5582,  2.1073, -0.7450, -0.0034,  1.5202, -1.3243,
          -0.9366,  2.6180, -0.6339]],

        [[ 0.5543,  0.5932,  0.1466,  0.2104, -0.2425,  0.0335, -0.2221,
           0.9109,  0.3075, -0.5472],
         [ 2.1602, -2.2460,  0.1286,  0.3825,  0.0081,  1.1463,  0.5968,
           3.0428,  0.0080, -2.1197],
         [-0.0711, -2.6423,  1.9394, -0.4242,  0.0202, -2.4865,  1.3386,
           1.6613,  1.0677,  1.8485]],

        [[-0.8536, -1.0123, -0.1331, -0.4215, -0.3352, -0.3252,  0.4758,
           0.3469,  0.3074, -0.3635],
         [ 0.5422,  0.9570, -0.3695, -0.3208,  1.7008, -1.5595,  0.3833,
          -0.9529,  0.9771, -1.1279],
         [ 1.3337, -1.8153, -1.0798,  0.7307, -0.9975, -1.1125,  1.3222,
          -0.7713, -0.

models

In [ ]:
class DotProductAttention(nn.Module):
    def __init__(self, dropout, **kwargs):
        super(DotProductAttention, self).__init__(**kwargs)
        self.dropout = nn.Dropout(dropout)
        # Shape of `queries`: (`batch_size`, no. of queries, `d`)
        # Shape of `keys`: (`batch_size`, no. of key-value pairs, `d`)
        # Shape of `values`: (`batch_size`, no. of key-value pairs, value
        # dimension)
        # Shape of `valid_lens`: (`batch_size`,) or (`batch_size`, no. of queries)
        
    def forward(self, queries, keys, values):
        d = queries.shape[-1]
        # Set `transpose_b=True` to swap the last two dimensions of `keys`
        scores = torch.bmm(queries, keys.transpose(1,2)) / math.sqrt(d)
        return torch.bmm(self.dropout(scores), values)

In [ ]:
import argparse
import torch
import torch.nn as nn

'''

input_size = 10
hidden_size = 20
num_layers = 2
dropout = 0.5

'''
class AttentionGRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout,sequence_length,classes, **kwargs ):
        super(AttentionGRU,self).__init__(**kwargs)
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.sequence_length = sequence_length 
        self.num_layers = num_layers
        self.dropout = dropout
        self.classes = classes
        # -> x needs to be: (batch_size, seq, input_size) #rnn = nn.GRU(10, 20, 2)
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, bidirectional=False)
        self.attention =  DotProductAttention(dropout)
        self.fc = nn.Linear(hidden_size*sequence_length,classes)

    
    def forward(self,X):
        h0 = torch.randn(2, 5, 20) #h0 = torch.randn(2, 3, 20)
        output, hn = self.gru(X, h0)
        output = self.attention(output,output,output)
        output = output.view(5, -1)
        output = self.fc(output)
    
        return output  

#train models


In [72]:
#二分类

'''
loss损失函数
nn.BCEWithLogitsLoss()二分类
nn.CrossEntropyLoss()多分类
https://blog.csdn.net/weixin_35757704/article/details/118388424

'''
#loss_function = nn.CrossEntropyLoss()
loss_function = nn.BCEWithLogitsLoss()
learning_rate = 1e-2
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [73]:
#二分类
total_train_step = 0
epoch = 10
for i in  range(epoch):
    model.train()
    for i,(x,target) in enumerate(train_loader):
        #expected scalar type Double but found Float
        
        x = x.to(torch.float32)
        #print(type(x),x)
        target = target.to(torch.float32)
        #print(type(target), target)
        outputs = model(x)
        #print(type(outputs), outputs)
        
        loss = loss_function(outputs,target)
        
        #优化器优化模型参数
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_train_step = total_train_step + 1
        if total_train_step % 100 == 0:
            print("训练次数：{}, Loss: {}".format(total_train_step, loss.item()))  

训练次数：100, Loss: -15609.322265625
训练次数：200, Loss: -23130.73828125


datalader dataset
1.https://shashikachamod4u.medium.com/excel-csv-to-pytorch-dataset-def496b6bcc1
2.https://colab.research.google.com/github/biswajitsahoo1111/blog_notebooks/blob/master/Reading_multiple_csv_files_in_PyTorch.ipynb
3.
4.
5.

In [ ]:
import pandas as pd
import torch 
from torch.utils.data import Dataset 
from sklearn.preprocessing import StandardScaler

class FeatureDataset(Dataset):
    def __init__(self,file_name):
        
        #read csv file and load raw data into variables
        file_out = pd.read_csv(file_name)
        x = file_out.iloc[1:216,1:12].values
        y = file_out[1:216,12].values
        
        #Feature Scaling
        sc = StandardScaler()
        x_train = sc.fit_transform(x)
        y_train = y
        
        #converting to torch tensors
        self.X_train = torch.tensor(x_train, dtype = torch.float32)
        self.y_train = torch.tensor(y_train)
        
    def __len__(self):
        return len(self.y_train)
    
    def __getitem__(self,idx):
        return self.X_train[idx],self.X_train[idx]